After familiarizing yourself with the syntax of rgxlog, <br>
let's explore another example that highlights the power of this declarative language. <br>
given a story.txt file which contains in between its lines parent-child information, the information is provided as follows:
X is the parent/father of Y
Y is the child/daughter of X

The story also has some last names.
The task is to identify the last name of all individuals.
It's important to note that you don't have access to individual last names, but you can deduce them by constructing family trees based on the information provided in the story.

Take a couple of minutes to think a solution for this problem.

---

Now that you understand the complexity of deducing last names based on family trees when using Python and extracting the parent-child relationships using Python, <br>
let us show you how easily we can do that by incorporating rgxlog into the equation.


In [ ]:
import rgxlog

Installation NLP failed
cargo or rustup are not installed in $PATH. please install rust: https://rustup.rs/


In [ ]:
%%rgxlog
story = read("sample_data/story.txt")
parent(X,Y) <- py_rgx_string(story, "(\w+)\s+is\s+the\s+daughter\s+of\s+(\w+)") -> (Y,X)
parent(X,Y) <- py_rgx_string(story, "(\w+)\s+is\s+the\s+child\s+of\s+(\w+)") -> (Y,X)
parent(X,Y) <- py_rgx_string(story, "(\w+)\s+is\s+the\s+parent\s+of\s+(\w+)") -> (X,Y)
parent(X,Y) <- py_rgx_string(story, "(\w+)\s+is\s+the\s+father\s+of\s+(\w+)") -> (X,Y)
lastname(X,Y) <- py_rgx_string(story, "([A-Z][a-z]+)\s([A-Z][a-z]+)") -> (X,Y)
?parent(X,Y)
?lastname(X,Y)

printing results for query 'parent(X, Y)':
    X     |    Y
----------+----------
   Aria   |   Mila
 Benjamin |  Daniel
 Benjamin |   Lila
   Owen   |   Aria
   Owen   |  Caleb
   Owen   |  Eliana
   Yaer   | Benjamin

printing results for query 'lastname(X, Y)':
    X     |   Y
----------+--------
   Aria   | Walker
 Benjamin | Smith



## Deducing Last Names Based on Family Trees with rgxlog

The following rgxlog code provides a simple yet powerful way to deduce the last name of each person based on the family tree information extracted to the parent relation.

In [ ]:
%%rgxlog
Family(X,Y) <- lastname(Y,X)
Family(X,Y) <- Family(X,Z), parent(Y,Z)
Family(X,Y) <- Family(X,Z), parent(Z,Y)
?Family(X,Y)

printing results for query 'Family(X, Y)':
   X    |    Y
--------+----------
 Smith  | Benjamin
 Smith  |  Daniel
 Smith  |   Lila
 Smith  |   Yaer
 Walker |   Aria
 Walker |  Caleb
 Walker |  Eliana
 Walker |   Mila
 Walker |   Owen

